# Analysis of unsupervised models

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

ground_truth_file = utils.join_paths(signals_dir, 'respiration_gt.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal_uv'] = predictions['signal_uv'].apply(eval).apply(np.array)
predictions['signal_uv_std'] = predictions['signal_uv_std'].apply(eval).apply(np.array)
predictions['signal_v'] = predictions['signal_v'].apply(eval).apply(np.array)
predictions['signal_v_std'] = predictions['signal_v_std'].apply(eval).apply(np.array)
predictions.head()

## Get the BPM for each prediction and ground truth signal

In [ ]:
import os
from tqdm.auto import tqdm
import respiration.analysis as analysis

analysis_dir = utils.dir_path('outputs', 'analysis', mkdir=True)
analysis_file = utils.join_paths(analysis_dir, 'raft_analysis.csv')

signal_directions = [
    'signal_uv',  # Amplitudes of the motion vectors in the horizontal and vertical directions
    'signal_v',  # Amplitudes of the motion vectors in the vertical direction
]

if os.path.exists(analysis_file):
    analysis_results = pd.read_csv(analysis_file)
else:
    analysis_results = []

    for index, row in tqdm(predictions.iterrows(), total=len(predictions)):
        subject, setting = row['subject'], row['setting']

        # Get the ground truth signal and sampling rate
        gt_signal = ground_truth[
            (ground_truth['subject'] == subject) &
            (ground_truth['setting'] == setting)
            ]['signal'].values[0]

        for signal_direction in signal_directions:

            prediction, sampling_rate = row[signal_direction], row['sampling_rate']
            comparator = analysis.SignalComparator(
                prediction,
                gt_signal,
                sampling_rate,
            )

            for metric, result in comparator.all_results().items():
                analysis_results.append({
                    'subject': subject,
                    'setting': setting,
                    'model': row['model'],
                    'roi': row['roi'],
                    'signal_direction': signal_direction,
                    'metric': metric,
                    'prediction': result['prediction'],
                    'ground_truth': result['ground_truth'],
                })

    analysis_results = pd.DataFrame(analysis_results)
    analysis_results.to_csv(analysis_file, index=False)

In [ ]:
analysis_results.head()

## Get the mean and standard deviation of the results

In [ ]:
method_results = []

metrics = analysis_results['metric'].unique()
models = analysis_results['model'].unique()
rois = analysis_results['roi'].unique()

for model in models:
    for metric in metrics:
        for signal_direction in signal_directions:
            for roi in rois:
                data = analysis_results[
                    (analysis_results['model'] == model) &
                    (analysis_results['metric'] == metric) &
                    (analysis_results['roi'] == roi) &
                    (analysis_results['signal_direction'] == signal_direction)
                    ]

                error = (data['prediction'] - data['ground_truth']).abs()

                method_results.append({
                    'model': model,
                    'metric': metric,
                    'roi': roi,
                    'signal_direction': signal_direction,
                    'mean_error': error.mean(),
                    'std_error': error.std(),
                })

method_results = pd.DataFrame(method_results)

In [ ]:
method_results

In [ ]:
# Group the results by signal direction. Calculate the mean and standard deviation of the error for each metric
grouped_results = method_results.groupby(['signal_direction', 'roi']).agg({
    'mean_error': 'mean',
    'std_error': 'mean',
}).reset_index()
grouped_results